# Sample from Spectrogram Feature Distribution

This notebook will help us sample the audio clips on various points of a spectrogram feature distribution. These spectrogram distributions should broadly represent things like loudness, pitch, speed, pausing, snr, etc.

In [28]:
import sys

# Setup the "PYTHONPATH"
sys.path.insert(0, '../../')

In [42]:
from src.datasets import m_ailabs_speech_dataset
from src.datasets import compute_spectrograms
from src.hparams import set_hparams
from random import sample 

set_hparams()
train, dev = m_ailabs_speech_dataset(directory='../../data/M-AILABS')
dev = sample(dev, 150)
dev = compute_spectrograms(dev, on_disk=False)

INFO:src.datasets.m_ailabs:Loading M-AILABS speech dataset
INFO:src.datasets.m_ailabs:Normalizing text and audio...
  5%|▌         | 2460/46297 [00:00<00:05, 7999.19it/s]WARNING:src.datasets.m_ailabs:Not found audio file, skipping: ../../data/M-AILABS/en_US/by_book/female/judy_bieber/the_master_key/wavs/the_master_key_05_f000135.wav

 36%|███▌      | 16550/46297 [00:02<00:03, 8725.23it/s]WARNING:src.datasets.m_ailabs:Not found audio file, skipping: ../../data/M-AILABS/en_US/by_book/female/mary_ann/midnight_passenger/wavs/midnight_passenger_05_f000269.wav

 44%|████▎     | 20238/46297 [00:02<00:02, 8706.81it/s]WARNING:src.datasets.m_ailabs:Text is absent, skipping: ../../data/M-AILABS/en_US/by_book/female/mary_ann/northandsouth/wavs/northandsouth_06_f000010.wav

 56%|█████▌    | 25992/46297 [00:03<00:02, 9142.13it/s]WARNING:src.datasets.m_ailabs:Not found audio file, skipping: ../../data/M-AILABS/en_US/by_book/female/mary_ann/northandsouth/wavs/northandsouth_40_f000069.wav

100%|███████

In [43]:
# Learn more: https://en.wikipedia.org/wiki/Speech_tempo
# Learn more: https://pdfs.semanticscholar.org/6ca5/9d11df13a78bf58d866f8712890d3220bc84.pdf
# TODO: Consider extracting phoneme's directly from the audio instead of the text.

# Install
# $ python3 -m pip install git+git://github.com/bootphon/phonemizer@master
# $ sudo apt-get install espeak
import phonemizer 
from phonemizer.phonemize import phonemize 
import re

seperator = phonemizer.separator.Separator(phone='-', syllable='', word='|')
    
def textToPhonemes(text, language):
    """
    Convert graphemes to phonemes.
    """
    ph = phonemize(text, separator=seperator, strip=False, njobs=1, backend='espeak', language=language)
    return ph

# Test case
textToPhonemes('Hello World. Hi There.', 'en')

INFO:root:initializing backend espeak-1.48.03


'h-ə-l-əʊ-|w-ɜː-l-d-|\nh-aɪ-|ð-eə-|'

In [66]:
import torch
# Install:
# $ python3 -m pip install git+https://github.com/csteinmetz1/pyloudnorm
import pyloudnorm 
import librosa
# Install:
# $ python3 -m pip install pyreaper
import pyreaper
import numpy

from tqdm import tqdm_notebook

signals = []
for row in tqdm_notebook(dev):
    """
    Compute signal features accross all the examples in ``dev``.
    """
    signal, sample_rate = librosa.core.load(str(row.audio_path), sr=None)
    meter = pyloudnorm.Meter(sample_rate)
    padded_signal = row.spectrogram_audio # Used to align f0 with spectrogram
    
    if signal.shape[0] < 10000:
        print('Too short for ``pyloudnorm``, skipping row:', row)
        continue
    
    # Learn more: 
    # https://www.tcelectronic.com/brand/tcelectronic/loudness-explained#googtrans(en|en)
    
    _, _, _, f0, _ = pyreaper.reaper((padded_signal * 32767).numpy().astype(numpy.int16), sample_rate)
    
    # Compute ``signal_to_noise_ratio``
    f0_to_spectrogram_index = lambda i: int(round((i / (f0.shape[0] - 1)) * (row.spectrogram.shape[0] - 1)))
    magnitude_spectrogram = numpy.exp(row.spectrogram.numpy())
    silence_energy = numpy.median([numpy.sum(magnitude_spectrogram[f0_to_spectrogram_index(i)])
                              for i, val in enumerate(f0) if val <= 0])
    other_energy = numpy.median([numpy.sum(magnitude_spectrogram[f0_to_spectrogram_index(i)])
                              for i, val in enumerate(f0) if val > 0])
    signal_to_noise_ratio = (other_energy - silence_energy) / silence_energy
    
    loudness = meter.integrated_loudness(signal)
    phonemes = textToPhonemes(row.text, 'en')
    
    signals.append({
        'audio_path': row.audio_path,
        'loudness': loudness,
        'samples-per-character': signal.shape[0] / len(row.text),
        'samples-per-phoneme': signal.shape[0] / phonemes.count('-'),
        'samples-per-word': signal.shape[0] / phonemes.count('|'),
        'f0': numpy.median(f0[f0 > 0]), # Average Pitch
        'silence': numpy.where(f0 <=0)[0].shape[0] / f0.shape[0], # Percentage Silence
        'signal_to_noise_ratio': signal_to_noise_ratio, # Signal to noise ratio
    })

INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend espeak-1.48.03
INFO:root:initializing backend esp

In [ ]:
from IPython.display import Audio, display
from random import sample 
import os

for row in sorted(signals, key=lambda r: r['samples-per-phoneme']):
    """
    Display features to the user in a sorted order; helping visualize the feature distribution.
    """
    print('audio_path:', row['audio_path'])
    print('loudness:', row['loudness'])
    print('samples-per-character:', row['samples-per-character'])
    print('samples-per-phoneme:', row['samples-per-phoneme'])
    print('samples-per-word:', row['samples-per-word'])
    print('f0:', row['f0'])
    print('silence:', row['silence'])
    print('signal_to_noise_ratio:', row['signal_to_noise_ratio'])
    display(Audio(str(row['audio_path'])))